In [159]:
import sys  
sys.path.insert(0, '..')

from config import Config

import pandas as pd
from sqlalchemy import create_engine

db = create_engine(Config.SQLALCHEMY_DATABASE_URI)

In [193]:
mista = pd.read_sql_query(
    """
    select ockovaci_mista.id ockovaci_misto_id, ockovaci_mista.nazev, okres_id, kraj_id 
    from ockovaci_mista join okresy on ockovaci_mista.okres_id=okresy.id;
    """,
    db
)
# mista

In [195]:
df = pd.read_sql_query("select * from ockovani_distribuce limit 10;", db)
# df

In [196]:
prijato = pd.read_sql_query(
    """
    select ockovaci_misto_id, vyrobce, sum(pocet_davek) prijato
    from ockovani_distribuce 
    where akce = 'Příjem' 
    group by (ockovaci_misto_id, vyrobce);
    """,
    db
)
# prijato

In [197]:
prijato_odjinud = pd.read_sql_query(
    """
    select cilove_ockovaci_misto_id ockovaci_misto_id, vyrobce, sum(pocet_davek) prijato_odjinud
    from ockovani_distribuce 
    where akce = 'Výdej' 
    group by (cilove_ockovaci_misto_id, vyrobce);
    """,
    db
)
# prijato_odjinud

In [198]:
vydano = pd.read_sql_query(
    """
    select ockovaci_misto_id, vyrobce, sum(pocet_davek) vydano
    from ockovani_distribuce 
    where akce = 'Výdej' 
    group by (ockovaci_misto_id, vyrobce);
    """,
    db
)
# vydano

In [199]:
df = pd.read_sql_query("select * from ockovani_spotreba limit 10;", db)
# df

In [200]:
spotreba = pd.read_sql_query(
    """
    select ockovaci_misto_id, vyrobce, sum(pouzite_davky) pouzito, sum(znehodnocene_davky) znehodnoceno
    from ockovani_spotreba 
    group by (ockovaci_misto_id, vyrobce);
    """,
    db
)
# spotreba

In [238]:
mista_key = mista
mista_key['join'] = 0

vyrobci_key = spotreba[['vyrobce']].drop_duplicates()
vyrobci_key['join'] = 0

merged = mista_key.merge(vyrobci_key).drop('join', axis=1)

merged = pd.merge(merged, prijato, how="left")
merged = pd.merge(merged, prijato_odjinud, how="left")
merged = pd.merge(merged, vydano, how="left")
merged = pd.merge(merged, spotreba, how="left")

merged['prijato'] = merged['prijato'].fillna(0).astype('int')
merged['prijato_odjinud'] = merged['prijato_odjinud'].fillna(0).astype('int')
merged['vydano'] = merged['vydano'].fillna(0).astype('int')
merged['pouzito'] = merged['pouzito'].fillna(0).astype('int')
merged['znehodnoceno'] = merged['znehodnoceno'].fillna(0).astype('int')
merged['skladem'] = merged['prijato'] + merged['prijato_odjinud'] - merged['vydano'] - merged['pouzito'] - merged['znehodnoceno']

merged

ockovaci_misto_id  \
0    63c9078b-0701-4da0-821e-4eaa0d9c9968   
1    63c9078b-0701-4da0-821e-4eaa0d9c9968   
2    63c9078b-0701-4da0-821e-4eaa0d9c9968   
3    989b3700-401f-4700-a684-ae09d348a3c0   
4    989b3700-401f-4700-a684-ae09d348a3c0   
..                                    ...   
829  98c76ec8-f7d1-474a-927c-d4a4fb2dcd0a   
830  98c76ec8-f7d1-474a-927c-d4a4fb2dcd0a   
831  cb0f8a24-3c54-467e-9e67-de01f424ea85   
832  cb0f8a24-3c54-467e-9e67-de01f424ea85   
833  cb0f8a24-3c54-467e-9e67-de01f424ea85   

                                                 nazev okres_id kraj_id  \
0    Krajská zdravotní a.s., - MNUL pracoviště výst...   CZ0423   CZ042   
1    Krajská zdravotní a.s., - MNUL pracoviště výst...   CZ0423   CZ042   
2    Krajská zdravotní a.s., - MNUL pracoviště výst...   CZ0423   CZ042   
3                                   Nemocnice Hořovice   CZ0202   CZ020   
4                                   Nemocnice Hořovice   CZ0202   CZ020   
..                                                 ...      ...     ...   
829                                  ÚVN Praha - sklad   CZ0100   CZ010   
830                                  ÚVN Praha - sklad   CZ0100   CZ010   
831                 Česká Lípa očkovací místo  Crystal   CZ0511   CZ051   
832                 Česká Lípa očkovací místo  Crystal   CZ0511   CZ051   
833                 Česká Lípa očkovací místo  Crystal   CZ0511   CZ051   

         vyrobce  prijato  prijato_odjinud  vydano  pouzito  znehodnoceno  \
0        Moderna        0                0       0        0             0   
1         Pfizer        0                0       0        0             0   
2    AstraZeneca        0                0       0        0             0   
3        Moderna      900                0      70      530             0   
4         Pfizer        0             4068     150     3066            30   
..           ...      ...              ...     ...      ...           ...   
829       Pfizer    32760             2340   22518     8694            18   
830  AstraZeneca        0                0       0        0             0   
831      Moderna        0                0       0        0             0   
832       Pfizer        0             2874       0     3114             0   
833  AstraZeneca        0              400       0      400             0   

     skladem  
0          0  
1          0  
2          0  
3        300  
4        822  
..       ...  
829     3870  
830        0  
831        0  
832     -240  
833        0  

[834 rows x 11 columns]